<a href="https://colab.research.google.com/github/AminRoma/Eluvio_Challange/blob/main/EluvioChallangeCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pickle
import pandas as pd
import os
import base64
import csv
import numpy as np
from sklearn.metrics import average_precision_score
from google.colab import drive
import sys
import glob
import json
from glob import glob
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
def calc_ap(gt_dict, pr_dict):
    """Average Precision (AP) for scene transitions.
    Args:
        gt_dict: Scene transition ground-truths.
        pr_dict: Scene transition predictions.
    Returns:
        AP, mean AP, and a dict of AP for each movie.
    """
    assert gt_dict.keys() == pr_dict.keys()

    AP_dict = dict()
    gt = list()
    pr = list()
    for imdb_id in gt_dict.keys():
        AP_dict[imdb_id] = average_precision_score(gt_dict[imdb_id], pr_dict[imdb_id])
        gt.append(gt_dict[imdb_id])
        pr.append(pr_dict[imdb_id])

    mAP = sum(AP_dict.values()) / len(AP_dict)

    gt = np.concatenate(gt)
    pr = np.concatenate(pr)
    AP = average_precision_score(gt, pr)

    return AP, mAP, AP_dict


In [34]:
def calc_miou(gt_dict, pr_dict, shot_to_end_frame_dict, threshold=0.5):
    """Maximum IoU (Miou) for scene segmentation.
    Miou measures how well the predicted scenes and ground-truth scenes overlap. The descriptions can be found in
    https://arxiv.org/pdf/1510.08893.pdf. Note the length of intersection or union is measured by the number of frames.
    Args:
        gt_dict: Scene transition ground-truths.
        pr_dict: Scene transition predictions.
        shot_to_end_frame_dict: End frame index for each shot.
        threshold: A threshold to filter the predictions.
    Returns:
        Mean MIoU, and a dict of MIoU for each movie.
    """
    def iou(x, y):
        s0, e0 = x
        s1, e1 = y
        smin, smax = (s0, s1) if s1 > s0 else (s1, s0)
        emin, emax = (e0, e1) if e1 > e0 else (e1, e0)
        return (emin - smax + 1) / (emax - smin + 1)

    def scene_frame_ranges(scene_transitions, shot_to_end_frame):
        end_shots = np.where(scene_transitions)[0]
        scenes = np.zeros((len(end_shots) + 1, 2), dtype=end_shots.dtype)
        scenes[:-1, 1] = shot_to_end_frame[end_shots]
        scenes[-1, 1] = shot_to_end_frame[len(scene_transitions)]
        scenes[1:, 0] = scenes[:-1, 1] + 1
        return scenes

    def miou(gt_array, pr_array, shot_to_end_frame):
        gt_scenes = scene_frame_ranges(gt_array, shot_to_end_frame)
        pr_scenes = scene_frame_ranges(pr_array >= threshold, shot_to_end_frame)
        assert gt_scenes[-1, -1] == pr_scenes[-1, -1]

        m = gt_scenes.shape[0]
        n = pr_scenes.shape[0]

        # IoU for (gt_scene, pr_scene) pairs
        iou_table = np.zeros((m, n))

        j = 0
        for i in range(m):
            # j start prior to i end
            while pr_scenes[j, 0] <= gt_scenes[i, 1]:
                iou_table[i, j] = iou(gt_scenes[i], pr_scenes[j])
                if j < n - 1:
                    j += 1
                else:
                    break
            # j end prior to (i + 1) start
            if pr_scenes[j, 1] < gt_scenes[i, 1] + 1:
                break
            # j start later than (i + 1) start
            if pr_scenes[j, 0] > gt_scenes[i, 1] + 1:
                j -= 1
        assert np.isnan(iou_table).sum() == 0
        assert iou_table.min() >= 0

        # Miou
        return (iou_table.max(axis=0).mean() + iou_table.max(axis=1).mean()) / 2

    assert gt_dict.keys() == pr_dict.keys()

    miou_dict = dict()
    for imdb_id in gt_dict.keys():
         miou_dict[imdb_id] = miou(gt_dict[imdb_id], pr_dict[imdb_id], shot_to_end_frame_dict[imdb_id])
    mean_miou = sum(miou_dict.values()) / len(miou_dict)

    return mean_miou, miou_dict

In [35]:
def calc_precision_recall(gt_dict, pr_dict, threshold=0.5):
    """Precision, Recall and F1 for scene transitions at a given threshold.
    Args:
        gt_dict: Scene transition ground-truths.
        pr_dict: Scene transition predictions.
        threshold: A threshold to filter the predictions.
    Returns:
        Mean Precision, Recall, and F1, per IMDB ID Precisions, Recalls, and F1 scores.
    """
    def precision_recall(gt_array, pr_array):
        tp_fn = gt_array == 1
        tp_fp = pr_array >= threshold

        tps = (tp_fn & tp_fp).sum()

        precision = tps / tp_fp.sum()
        recall = tps / tp_fn.sum()

        return np.nan_to_num(precision), np.nan_to_num(recall)

    assert gt_dict.keys() == pr_dict.keys()

    precision_dict = dict()
    recall_dict = dict()
    fscore_dict = dict()

    for imdb_id in gt_dict.keys():
        p, r = precision_recall(gt_dict[imdb_id], pr_dict[imdb_id])
        precision_dict[imdb_id] = p
        recall_dict[imdb_id] = r
        fscore_dict[imdb_id] = 2 * p * r / (p + r)

    n = len(gt_dict)
    mean_precision = sum(precision_dict.values()) / n
    mean_recall = sum(recall_dict.values()) / n
    mean_fscore = sum(fscore_dict.values()) / n

    return mean_precision, mean_recall, mean_fscore, precision_dict, recall_dict, fscore_dict


In [36]:
data_folder = os.path.join('/content/drive/MyDrive/Colab Notebooks/EluvioData/Eluvio')
filenames = glob(os.path.join(data_folder, '*.pkl'))
gt_dict = dict()
pr_dict = dict()
shot_to_end_frame_dict = dict()
scores = dict()

for fn in filenames:
    x = pickle.load(open(fn, "rb"))


    gt_dict[x["imdb_id"]] = x["scene_transition_boundary_ground_truth"]
    pr_dict[x["imdb_id"]] = x["scene_transition_boundary_prediction"]
    shot_to_end_frame_dict[x["imdb_id"]] = x["shot_end_frame"]

    p1 = pd.DataFrame(x['scene_transition_boundary_ground_truth']).astype("float")  
    p2 = pd.DataFrame(x['scene_transition_boundary_prediction']).astype("float")  
    p3 = pd.DataFrame(x['place']).astype("float")  
    p4 = pd.DataFrame(x['cast']).astype("float")  
    p5 = pd.DataFrame(x['action']).astype("float")  
    p6 = pd.DataFrame(x['audio']).astype("float")


    p = pd.concat([p3, p4,p5, p6, p1], axis=1, ignore_index=True)
    p = p.head(-1)

    X = p.iloc[ : , :-1].values
    y = p.iloc [: , -1].values

    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    imputer.fit(X[:, :])
    X = imputer.transform(X[:, :])
    sc = StandardScaler()
    X[:,:] = sc.fit_transform(X[:,:])
    kpca = KernelPCA(n_components=20, kernel = 'rbf')
    X = kpca.fit_transform(X)
    regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
  # regressor = SVR(kernel ='rbf')
  # regressor = DecisionTreeRegressor(random_state=0)

    regressor.fit(X, y)
    y_pred = regressor.predict(X)
    y_pred = pd.DataFrame(y_pred)
    pr_dict[x["imdb_id"]] = y_pred
    y_pred.to_csv(fn+".csv", sep=',', index=False)

    scores["AP"], scores["mAP"], _ = calc_ap(gt_dict, pr_dict)
    scores["Miou"], _ = calc_miou(gt_dict, pr_dict, shot_to_end_frame_dict)

    print("Scores:", json.dumps(scores, indent=4))


Scores: {
    "AP": 0.998602448947112,
    "mAP": 0.998602448947112,
    "Miou": 0.9795657170365565
}
Scores: {
    "AP": 0.9990572330799564,
    "mAP": 0.999301224473556,
    "Miou": 0.9831233736918855
}
Scores: {
    "AP": 0.9981523649230603,
    "mAP": 0.9984087935236813,
    "Miou": 0.9782818679990207
}
Scores: {
    "AP": 0.9973650733998687,
    "mAP": 0.9961509174870833,
    "Miou": 0.9718742820174903
}
Scores: {
    "AP": 0.9964508384174533,
    "mAP": 0.9959321492606785,
    "Miou": 0.9686898322178331
}
Scores: {
    "AP": 0.9961331844600347,
    "mAP": 0.9953559437818921,
    "Miou": 0.9655229918008296
}
Scores: {
    "AP": 0.9960314127506149,
    "mAP": 0.9955400028066761,
    "Miou": 0.9625149115464959
}
Scores: {
    "AP": 0.9956370121161046,
    "mAP": 0.9953628689798097,
    "Miou": 0.9600249517099162
}
Scores: {
    "AP": 0.9960450276187283,
    "mAP": 0.9958018281091826,
    "Miou": 0.9628756129265168
}
Scores: {
    "AP": 0.9957224990151919,
    "mAP": 0.99525509400498

In [37]:
y_pred

,0
0,0.1
1,0.2
2,0.1
3,0.1
4,0.1
...,...
2543,0.0
2544,0.0
2545,0.0
2546,0.5
